<a href="https://colab.research.google.com/github/FMurray/hyperdemocracy/blob/main/hyper_democracy_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
# if you are on a google colab, uncomment the lines below to fetch the requirements file and the hyperdemocracy.py module
# and pip install the requirements

#!wget https://raw.githubusercontent.com/FMurray/hyperdemocracy/main/requirements.txt
#!wget https://raw.githubusercontent.com/FMurray/hyperdemocracy/main/hyperdemocracy.py
#!pip install -r requirements.txt

In [ ]:
import os
import rich

# Note on Formatted Output

Note that we patch the builtin Python `print` function with `rich.print` in the cell below. If you prefer a more traditional print output you can comment out the import below. 

In [ ]:
from rich import print

# Note on Cost

Running this notebook with your OpenAI key in an environment variable will charge a small amount of money to your OpenAI account. The total cost of running this notebook multiple times should be less than 10 cents but that can change if the datasource is changed. Each cell that makes a request to an OpenAI endpoint that costs money will have the following comment in it, 

```
## THIS CELL SPENDS MONEY ##
```

Up to date pricing information on OpenAI models can be found here https://openai.com/pricing

# Setup Keys

In [ ]:
# if you want to use local secrets, add a file called .env to this directory and uncomment the lines below

from dotenv import load_dotenv
load_dotenv(".env")
#%dotenv ./.env

In [ ]:
# if you are using google colab, uncomment the lines below to manually enter your OpenAI key.

#import getpass
#os.environ['OPENAI_API_KEY'] = getpass.getpass()

In [ ]:
%load_ext autoreload
%autoreload 2

# Load Assembleco Records

We are going to use a small subset of records provided by https://assembled.app/.

For the purposes of this workshop, we have created a [huggingface dataset](https://huggingface.co/datasets/assembleco/hyperdemocracy)  which we can load using the `load_dataset` function. This is all handled for you in the `load_assembleco_records` function. See more info here [datasets](https://huggingface.co/docs/datasets/index) package.

In [ ]:
from hyperdemocracy import load_assembleco_records

In [ ]:
df = load_assembleco_records(process=True, strip_html=True, remove_empty_body=True)

In [ ]:
df.head()

In [ ]:
df.shape

# Sponsor Graph Sidequest

We will be focusing on the text content of the legislation in this workshop, but if you would like to explore building a graph from the sponsor / co-sponsor / legislation network check out the [sponsor_graph notebook](https://github.com/FMurray/hyperdemocracy/blob/main/sidequests/sponsor_graph.ipynb) to get started.

# From Pandas Dataframe to LangChain Documents

A langchain document is a simple class with two attributes, 
* page_content (a string)
* metadata (a dictionary)

In [ ]:
from langchain.schema import Document 

In [ ]:
Document??

Below we take each row from our legislation DataFrame and create a LangChain Document. We use the `body` column for the `page_content` attribute and populate the `metadata` attribute with data from some of the other columns. Note that the `source` key in the `metadata` dictionary is associated with a congress.gov url. The `source` key can hold an arbitrary string and will become important when we look into question answering systems that return information about the sources used to answer a question. We also restrict ourselves to `str`, `int`, and `float` types in the other values of our `metadata` dictionary. This is to make it easy to use them as filters when querying our vectorstore. If that doesn't make sense, dont worry! It will by the end of the workshop.  

In [ ]:
docs = []
for irow, row in df.iterrows():
    doc = Document(
        page_content=row['body'],
        metadata={
            # Note: chroma can only filter on float, str, or int
            # https://docs.trychroma.com/usage-guide#using-where-filters
            'key': row['key'],
            'congress_num': row['congress_num'],
            'legis_class': row['legis_class'],
            'legis_num': row['legis_num'],
            'name': row['name'],
            'summary': row['summary'],
            'sponsor': row['sponsors'][0][0],
            'source': row['congress_gov_url'],
        },
    )
    docs.append(doc)

In [ ]:
print(docs[0])

## Activity

* examine the Document content
* visit the congress.gov URL and view the document in various formats
* examine the body text below
* read the summary of the document and attempt to connect it with the long form text of the document
* see if there is anything interesting that can be done with this https://twitter.com/RLanceMartin/status/1670489431168659456?s=20

In [ ]:
print(docs[0].page_content)

In [ ]:
print(len(docs))

# Document QA Quickstart

* https://python.langchain.com/docs/modules/chains/additional/question_answering
* https://python.langchain.com/docs/modules/chains/document.html

Our goal is to setup a question answering (QA) system that can repond to natural language questions about legislation using source material that we provide. In the following section, we will explore the quickest most high-level approach provided by LangChain. Afterwards, we will unpack all of components and go over them in more detail. 

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
## THIS CELL SPENDS MONEY ##
index = VectorstoreIndexCreator().from_documents(docs)

In [ ]:
## THIS CELL SPENDS MONEY ##

# QA 

# copy paste this cell and try some questions of your own
query = "What are the primary themes around energy policy?"
out = index.query(query)
out

In [ ]:
## THIS CELL SPENDS MONEY ##

# QA with sources

# copy paste this cell and try some questions of your own
out = index.query_with_sources(query)
print("question:\n", out['question'])
print("answer:\n", out['answer'])
print("sources:\n", out['sources'])

## Activity

* Contemplate why the answers are slightly different between the "QA" result and the "QA with sources" result.
* Visit the source links and check if the linked legislation is relevant to the question.

# Document QA - Step by Step

Now that we've used the high-level tools of LangChain, lets go into more detail.

## Langchain Text Splitters

> When you want to deal with long pieces of text, it is necessary to split up that text into chunks. As simple as this sounds, there is a lot of potential complexity here. Ideally, you want to keep the semantically related pieces of text together. What "semantically related" means could depend on the type of text. This notebook showcases several ways to do that.

> At a high level, text splitters work as following:

>    1. Split the text up into small, semantically meaningful chunks (often sentences).
>    2. Start combining these small chunks into a larger chunk until you reach a certain size (as measured by some function).
>    3. Once you reach that size, make that chunk its own piece of text and then start creating a new chunk of text with some overlap (to keep context between chunks).

> That means there are two different axes along which you can customize your text splitter:

>    1. How the text is split
>    2. How the chunk size is measured

-- https://python.langchain.com/docs/modules/data_connection/document_transformers/#text-splitters

Here are some useful options for splitting legislative text, 

* [character text splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/character_text_splitter)
  * How the text is split: by single character
  * How the chunk size is measured: by number of characters
* [recursive text splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)
  * How the text is split: by list of characters
  * How the chunk size is measured: by number of characters
* [split by token](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/split_by_token)
  * How the text is split: by character passed in
  * How the chunk size is measured: by tiktoken tokenizer

If you are not familiar with the concept of a token, this article may help, 
* https://simonwillison.net/2023/Jun/8/gpt-tokenizers/

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter

In [ ]:
text = """We hold these truths to be self-evident, that all men are created equal,

that they are endowed by their Creator with certain unalienable Rights,

that among these are Life, Liberty and the pursuit of Happiness."""

### CharacterTextSplitter

In [ ]:
# this is the default separator
CharacterTextSplitter(separator="\n\n", chunk_size=20, chunk_overlap=0).split_text(text)

In [ ]:
# this is what happens if we change the default separator
CharacterTextSplitter(separator=" ", chunk_size=20, chunk_overlap=0).split_text(text)

In [ ]:
# this is what overlap does
CharacterTextSplitter(separator=" ", chunk_size=20, chunk_overlap=10).split_text(text)

### RecursiveCharacterTextSplitter

In [ ]:
# these are the default separators
RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", ""], chunk_size=40, chunk_overlap=0).split_text(text)

In [ ]:
RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ",", " ", ""], chunk_size=40, chunk_overlap=0).split_text(text)

### TokenTextSplitter

In [ ]:
# the length unit here is tokens not characters
TokenTextSplitter(model_name="text-embedding-ada-002", chunk_size=10, chunk_overlap=0).split_text(text)

In [ ]:
# the length unit here is tokens not characters
TokenTextSplitter(model_name="text-embedding-ada-002", chunk_size=10, chunk_overlap=4).split_text(text)

In [ ]:
import tiktoken

In [ ]:
enc = tiktoken.encoding_for_model("text-embedding-ada-002")
print(enc)

In [ ]:
print(enc.encode(text))

In [ ]:
tokens = [enc.decode_single_token_bytes(token) for token in enc.encode(text)]
tokens

### Lets Make a TextSplitter Choice here

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_docs = text_splitter.split_documents(docs)

In [ ]:
print("Number of original docs: ", len(docs))
print("Number of split docs: ", len(split_docs))

## Embed and Index Doc Chunks

Now we will embed and index the document chunks from the previous section. 
We have many choices when it comes to text embedding models and vector indexes. 
For this tutorial we will choose, 

* text embedding model: `text_embedding_ada_002`
* vector index:
  * https://www.trychroma.com
  * https://docs.trychroma.com/usage-guide#changing-the-distance-function
  * https://github.com/nmslib/hnswlib/tree/master#supported-distances
  * https://github.com/hwchase17/langchain/blob/master/langchain/vectorstores/chroma.py
  * https://github.com/hwchase17/langchain/blob/master/langchain/vectorstores/utils.py#L10

For a look at some of the top performing closed and open source text embedding models, check out the HuggingFace Massive Text Embedding Benchmark (MTEB), 
* https://huggingface.co/spaces/mteb/leaderboard
  
For a more detailed introduction to embeddings in general, see the embeddings notebook
* https://github.com/FMurray/hyperdemocracy/blob/main/sidequests/embeddings_v2.ipynb

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
embeddings.embed_documents([text])[0][0:10]

In [ ]:
## THIS CELL SPENDS MONEY ##
vec_store = Chroma.from_documents(split_docs, embeddings)

In [ ]:
vec_store

In [ ]:
ret_docs = vec_store.similarity_search_with_score(
    "nuclear power", 
    k=5, 
    filter={"source": "https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/17"},
)

print("number of returned docs: ", len(ret_docs))
for doc in ret_docs:
    print(doc)

### Build A Retrieval Chain for Question Answering


In [ ]:
import langchain
langchain.verbose = False

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [ ]:
retriever = vec_store.as_retriever(search_kwargs={'k':10})

In [ ]:
retriever

In [ ]:
llm = OpenAI()

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

In [ ]:
llm

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [ ]:
from langchain.chains.combine_documents.base import BaseCombineDocumentsChain

In [ ]:
BaseCombineDocumentsChain?

In [ ]:
qa.combine_documents_chain

# Questions

* what are the components of the RetrievalQA chain?
* what is the QA prompt?
* how would you modify the QA prompt?
* what is the difference between the following qa chain types?,
    * stuff
    * map_reduce
    * map_rerank
    * refine

# Resources

* https://github.com/hwchase17/langchain/tree/master/langchain/chains/retrieval_qa
* https://github.com/hwchase17/langchain/tree/master/langchain/chains/question_answering

In [ ]:
from IPython import display

In [ ]:
display.Image("https://python.langchain.com/assets/images/stuff-818da4c66ee17911bc8861c089316579.jpg", width=600)

In [ ]:
# WARNING! Do not commit the outputs of this cell if it contains your API key

rich.print(qa)

## How many ways can we print a prompt? 

In [ ]:
prompt_template = qa.combine_documents_chain.llm_chain.prompt
prompt_template

In [ ]:
print(prompt_template.template)

In [ ]:
import textwrap

In [ ]:
rich.print(prompt_template.format(context='[CONTEXT]', question='[QUESTION]'))

In [ ]:
answer = qa("What is the solution to climate change?")

In [ ]:
answer.keys()

In [ ]:
print(answer['result'])

In [ ]:
qaws = RetrievalQAWithSourcesChain.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [ ]:
# WARNING! Do not commit the outputs of this cell if it contains your API key
print(qaws)

In [ ]:
pt = qaws.combine_documents_chain.llm_chain.prompt

In [ ]:
print(pt.format(summaries='[SUMMARIES]', question='[QUESTION]'))

In [ ]:
answer = qaws("What is the solution to climate change?")

In [ ]:
answer.keys()

In [ ]:
print(answer['answer'])
print(answer['sources'])

# Prompt Construction Sidequest

# TODO

Try alternatives to stuff

Figure out how to pass all the options to the high level constructor. 

https://github.com/hwchase17/langchain/blob/master/langchain/indexes/vectorstore.py

In [ ]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, 
    embedding=OpenAIEmbeddings(),
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
)


Sticking this here to decide if we want to use this in the course content

https://xml.house.gov/

TODO: Sidequest on implementing a langchain document loader using this XML schema ^^^

https://www.everycrsreport.com/

# Lets make it a conversation

https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html

In [ ]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# TODO cover serializing the db to disk
db

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
qachat = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0), 
    db.as_retriever(), 
    memory=memory
)

In [ ]:
query = "What is the solution to climate change?"
answer = qachat(query)

In [ ]:
print(answer)

In [ ]:
follow_up = "How certain is the 350 number?"
result = qachat({"question": follow_up})

In [ ]:
print(result)